In [11]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.layers import SimpleRNN
from keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

## Подготовка датасета

In [12]:
data = pd.read_csv("file.csv", sep=',')
x = data['Lyric']
y = data['language']
encoder = LabelEncoder()
y = encoder.fit_transform(y)
data = pd.DataFrame(x)
data['language'] = y
print(data)
train, test = train_test_split(data, test_size=0.2, random_state=42)

                                                 Lyric  language
0    Tudo o que eu quero nessa vida,\r\nToda vida, ...         2
1    Meu coração\r\nSem direção\r\nVoando só por vo...         2
2    É de babaixá!\r\nÉ de balacubaca!\r\nÉ de baba...         2
3    Quando a chuva passar\r\n\r\nPra quê falar\r\n...         2
4    A minha sorte grande foi você cair do céu\r\nM...         2
..                                                 ...       ...
125  Que falta eu sinto de um bem\r\nQue falta me f...         2
126  Eu tô vendo\r\nVocê tá balançando\r\nTô sentin...         2
127  Olha quem chegou\r\nNesse calor eu sei que vou...         2
128  Meu amor olha só hoje o sol não apareceu\r\nÉ ...         2
129  Faço qualquer negócio pra te ver na avenida ou...         2

[130 rows x 2 columns]


In [13]:
x = train["Lyric"]
y = train["language"]
x = np.asarray(x).astype(np.str_)
y = np.asarray(y).astype(np.float32)
test_x = test["Lyric"]
test_y = test["language"]
test_x = np.asarray(test_x).astype(np.str_)
test_y = np.asarray(test_y).astype(np.float32)

## Посимвольная векторизация

In [14]:
vectorizer = tf.keras.layers.TextVectorization(split='character')

print('Training vectorizer')
vectorizer.adapt(list(train["Lyric"]))

Training vectorizer


## SimpleRNN модель

In [15]:
model = keras.Sequential()
model.add(vectorizer)
model.add(layers.Embedding(input_dim=100000, output_dim=8))
model.add(layers.SimpleRNN(4))
model.add(layers.Dense(100, activation = 'relu'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_3 (TextV  (None, None)             0         
 ectorization)                                                   
                                                                 
 embedding_2 (Embedding)     (None, None, 8)           800000    
                                                                 
 simple_rnn_2 (SimpleRNN)    (None, 4)                 52        
                                                                 
 dense_2 (Dense)             (None, 100)               500       
                                                                 
Total params: 800,552
Trainable params: 800,552
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.compile(keras.optimizers.Adam(0.01), keras.losses.SparseCategoricalCrossentropy(from_logits=True), ['acc'])
train_model = model.fit(x=x, y=y, batch_size=64, epochs = 10, validation_data=(test_x, test_y))

Epoch 1/10
2/2 [==============================] - 3s 891ms/step - loss: 4.5583 - acc: 0.0000e+00 - val_loss: 4.4957 - val_acc: 0.0385
Epoch 2/10
2/2 [==============================] - 1s 522ms/step - loss: 4.4801 - acc: 0.0192 - val_loss: 4.4139 - val_acc: 0.0769
Epoch 3/10
2/2 [==============================] - 1s 517ms/step - loss: 4.3980 - acc: 0.3654 - val_loss: 4.3293 - val_acc: 0.9615
Epoch 4/10
2/2 [==============================] - 1s 360ms/step - loss: 4.3147 - acc: 0.9327 - val_loss: 4.2390 - val_acc: 0.9615
Epoch 5/10
2/2 [==============================] - 1s 501ms/step - loss: 4.2202 - acc: 0.9327 - val_loss: 4.1402 - val_acc: 0.9615
Epoch 6/10
2/2 [==============================] - 1s 448ms/step - loss: 4.1203 - acc: 0.9327 - val_loss: 4.0336 - val_acc: 0.9615
Epoch 7/10
2/2 [==============================] - 1s 488ms/step - loss: 4.0138 - acc: 0.9327 - val_loss: 3.9215 - val_acc: 0.9615
Epoch 8/10
2/2 [==============================] - 1s 533ms/step - loss: 3.9013 - acc: 

## Пословная токенизация

In [17]:
vectorizer = tf.keras.layers.TextVectorization(split='whitespace')

print('Training vectorizer')
vectorizer.adapt(list(train["Lyric"]))

Training vectorizer


## SimpleRNN модель

In [18]:
model = keras.Sequential()
model.add(vectorizer)
model.add(layers.Embedding(input_dim=100000, output_dim=64, mask_zero=True))
model.add(layers.SimpleRNN(4))
model.add(layers.Dense(100, activation = 'tanh'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_4 (TextV  (None, None)             0         
 ectorization)                                                   
                                                                 
 embedding_3 (Embedding)     (None, None, 64)          6400000   
                                                                 
 simple_rnn_3 (SimpleRNN)    (None, 4)                 276       
                                                                 
 dense_3 (Dense)             (None, 100)               500       
                                                                 
Total params: 6,400,776
Trainable params: 6,400,776
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.compile(keras.optimizers.Adam(0.01), keras.losses.SparseCategoricalCrossentropy(from_logits=True), ['acc'])
train_model = model.fit(x=x, y=y, batch_size=64, epochs = 10, validation_data=(test_x, test_y))

Epoch 1/10
2/2 [==============================] - 2s 685ms/step - loss: 4.5951 - acc: 0.0096 - val_loss: 4.5474 - val_acc: 0.0385
Epoch 2/10
2/2 [==============================] - 1s 372ms/step - loss: 4.4359 - acc: 0.0577 - val_loss: 4.4615 - val_acc: 0.1923
Epoch 3/10
2/2 [==============================] - 0s 165ms/step - loss: 4.3055 - acc: 0.6346 - val_loss: 4.3548 - val_acc: 0.6154
Epoch 4/10
2/2 [==============================] - 0s 170ms/step - loss: 4.1613 - acc: 0.9231 - val_loss: 4.2296 - val_acc: 0.7308
Epoch 5/10
2/2 [==============================] - 0s 159ms/step - loss: 4.0033 - acc: 0.9615 - val_loss: 4.0918 - val_acc: 0.8462
Epoch 6/10
2/2 [==============================] - 0s 156ms/step - loss: 3.8381 - acc: 1.0000 - val_loss: 3.9542 - val_acc: 0.8462
Epoch 7/10
2/2 [==============================] - 0s 154ms/step - loss: 3.6783 - acc: 1.0000 - val_loss: 3.8176 - val_acc: 0.9231
Epoch 8/10
2/2 [==============================] - 0s 169ms/step - loss: 3.5317 - acc: 0.99

## LSTM

## Подготовка датасета

In [20]:
data = pd.read_csv("file.csv")
x = data['Lyric']
y = data['language']
encoder = LabelEncoder()
y = encoder.fit_transform(y)
data = pd.DataFrame(x)
data['language'] = y
print(data)
train, test = train_test_split(data, test_size=0.2, random_state=42)

                                                 Lyric  language
0    Tudo o que eu quero nessa vida,\r\nToda vida, ...         2
1    Meu coração\r\nSem direção\r\nVoando só por vo...         2
2    É de babaixá!\r\nÉ de balacubaca!\r\nÉ de baba...         2
3    Quando a chuva passar\r\n\r\nPra quê falar\r\n...         2
4    A minha sorte grande foi você cair do céu\r\nM...         2
..                                                 ...       ...
125  Que falta eu sinto de um bem\r\nQue falta me f...         2
126  Eu tô vendo\r\nVocê tá balançando\r\nTô sentin...         2
127  Olha quem chegou\r\nNesse calor eu sei que vou...         2
128  Meu amor olha só hoje o sol não apareceu\r\nÉ ...         2
129  Faço qualquer negócio pra te ver na avenida ou...         2

[130 rows x 2 columns]


In [21]:
x = train["Lyric"]
y = train["language"]
x = np.asarray(x).astype(np.str_)
y = np.asarray(y).astype(np.float32)
test_x = test["Lyric"]
test_y = test["language"]
test_x = np.asarray(test_x).astype(np.str_)
test_y = np.asarray(test_y).astype(np.float32)

## Посимвольная векторизация

In [22]:
vectorizer = tf.keras.layers.TextVectorization(split='character')

print('Training vectorizer')
vectorizer.adapt(list(train["Lyric"]))

Training vectorizer


## Однослойная LSTM-модель

In [23]:
model = keras.Sequential()
model.add(vectorizer)
model.add(layers.Embedding(input_dim=40000, output_dim=8, mask_zero=True))
model.add(layers.LSTM(4))
model.add(layers.Dense(100, activation = 'tanh'))
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_5 (TextV  (None, None)             0         
 ectorization)                                                   
                                                                 
 embedding_4 (Embedding)     (None, None, 8)           320000    
                                                                 
 lstm (LSTM)                 (None, 4)                 208       
                                                                 
 dense_4 (Dense)             (None, 100)               500       
                                                                 
Total params: 320,708
Trainable params: 320,708
Non-trainable params: 0
_________________________________________________________________


In [24]:
model.compile(keras.optimizers.Adam(0.1), keras.losses.SparseCategoricalCrossentropy(from_logits=True), ['acc'])
train_model = model.fit(x=x, y=y, batch_size=64, epochs = 10, validation_data=(test_x, test_y))

Epoch 1/10
2/2 [==============================] - 6s 2s/step - loss: 4.4711 - acc: 0.3558 - val_loss: 3.7116 - val_acc: 0.9615
Epoch 2/10
2/2 [==============================] - 2s 1s/step - loss: 3.5669 - acc: 0.9327 - val_loss: 3.0300 - val_acc: 0.9615
Epoch 3/10
2/2 [==============================] - 2s 858ms/step - loss: 2.9876 - acc: 0.9327 - val_loss: 2.8276 - val_acc: 0.9615
Epoch 4/10
2/2 [==============================] - 2s 878ms/step - loss: 2.8207 - acc: 0.9327 - val_loss: 2.7849 - val_acc: 0.9615
Epoch 5/10
2/2 [==============================] - 2s 906ms/step - loss: 2.7838 - acc: 0.9327 - val_loss: 2.7732 - val_acc: 0.9615
Epoch 6/10
2/2 [==============================] - 2s 873ms/step - loss: 2.7731 - acc: 0.9327 - val_loss: 2.7697 - val_acc: 0.9615
Epoch 7/10
2/2 [==============================] - 2s 906ms/step - loss: 2.7670 - acc: 0.9327 - val_loss: 2.7684 - val_acc: 0.9615
Epoch 8/10
2/2 [==============================] - 2s 722ms/step - loss: 2.7638 - acc: 0.9327 - v

## Пословная векторизация

In [27]:
vectorizer = tf.keras.layers.TextVectorization(split='whitespace')

print('Training vectorizer')
vectorizer.adapt(list(train["Lyric"]))

Training vectorizer


## Многослойная LSTM-модель

In [28]:
model = keras.Sequential()
model.add(vectorizer)
model.add(layers.Embedding(input_dim=40000, output_dim=64, mask_zero=True))
model.add(layers.LSTM(8, return_sequences=True))
model.add(layers.LSTM(4))
model.add(layers.Dense(100, activation = 'tanh'))
model.add(layers.Dense(100, activation = 'tanh'))
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_8 (TextV  (None, None)             0         
 ectorization)                                                   
                                                                 
 embedding_5 (Embedding)     (None, None, 64)          2560000   
                                                                 
 lstm_1 (LSTM)               (None, None, 8)           2336      
                                                                 
 lstm_2 (LSTM)               (None, 4)                 208       
                                                                 
 dense_5 (Dense)             (None, 100)               500       
                                                                 
 dense_6 (Dense)             (None, 100)               10100     
                                                      

In [29]:
model.compile(keras.optimizers.Adam(0.01), keras.losses.SparseCategoricalCrossentropy(from_logits=True), ['acc'])
train_model = model.fit(x=x, y=y, batch_size=64, epochs = 10, validation_data=(test_x, test_y))

Epoch 1/10
2/2 [==============================] - 9s 2s/step - loss: 4.5343 - acc: 0.3750 - val_loss: 4.1300 - val_acc: 0.9615
Epoch 2/10
2/2 [==============================] - 1s 308ms/step - loss: 3.9648 - acc: 0.9327 - val_loss: 3.4499 - val_acc: 0.9615
Epoch 3/10
2/2 [==============================] - 1s 374ms/step - loss: 3.3562 - acc: 0.9423 - val_loss: 3.0176 - val_acc: 0.9615
Epoch 4/10
2/2 [==============================] - 1s 354ms/step - loss: 2.9729 - acc: 0.9423 - val_loss: 2.8345 - val_acc: 0.9615
Epoch 5/10
2/2 [==============================] - 1s 339ms/step - loss: 2.8131 - acc: 0.9423 - val_loss: 2.7977 - val_acc: 0.9615
Epoch 6/10
2/2 [==============================] - 1s 328ms/step - loss: 2.7727 - acc: 0.9423 - val_loss: 2.7942 - val_acc: 0.9615
Epoch 7/10
2/2 [==============================] - 1s 340ms/step - loss: 2.7680 - acc: 0.9423 - val_loss: 2.7941 - val_acc: 0.9615
Epoch 8/10
2/2 [==============================] - 1s 362ms/step - loss: 2.7646 - acc: 0.9423 

## Пословная векторизация

In [30]:
vectorizer = tf.keras.layers.TextVectorization(split='whitespace')

print('Training vectorizer')
vectorizer.adapt(list(train["Lyric"]))

Training vectorizer


## Двунаправленая LSTM-модель

In [31]:
model = keras.Sequential()
model.add(vectorizer)
model.add(layers.Embedding(input_dim=40000, output_dim=16, mask_zero=True))
model.add(layers.Bidirectional(keras.layers.LSTM(8), merge_mode='concat')),
model.add(layers.Dense(100, activation = 'tanh'))
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_9 (TextV  (None, None)             0         
 ectorization)                                                   
                                                                 
 embedding_6 (Embedding)     (None, None, 16)          640000    
                                                                 
 bidirectional (Bidirectiona  (None, 16)               1600      
 l)                                                              
                                                                 
 dense_7 (Dense)             (None, 100)               1700      
                                                                 
Total params: 643,300
Trainable params: 643,300
Non-trainable params: 0
_________________________________________________________________


In [32]:
model.compile(keras.optimizers.Adam(0.01), keras.losses.SparseCategoricalCrossentropy(from_logits=True), ['acc'])
train_model = model.fit(x=x, y=y, batch_size=64, epochs = 20, validation_data=(test_x, test_y))

Epoch 1/20
2/2 [==============================] - 9s 3s/step - loss: 4.5841 - acc: 0.3558 - val_loss: 4.4741 - val_acc: 0.9615
Epoch 2/20
2/2 [==============================] - 1s 295ms/step - loss: 4.4184 - acc: 0.9423 - val_loss: 4.2264 - val_acc: 0.9615
Epoch 3/20
2/2 [==============================] - 1s 294ms/step - loss: 4.1089 - acc: 0.9327 - val_loss: 3.8159 - val_acc: 0.9615
Epoch 4/20
2/2 [==============================] - 1s 308ms/step - loss: 3.6912 - acc: 0.9327 - val_loss: 3.4206 - val_acc: 0.9615
Epoch 5/20
2/2 [==============================] - 1s 284ms/step - loss: 3.3323 - acc: 0.9327 - val_loss: 3.1417 - val_acc: 0.9615
Epoch 6/20
2/2 [==============================] - 1s 506ms/step - loss: 3.0890 - acc: 0.9327 - val_loss: 2.9755 - val_acc: 0.9615
Epoch 7/20
2/2 [==============================] - 1s 592ms/step - loss: 2.9455 - acc: 0.9327 - val_loss: 2.8823 - val_acc: 0.9615
Epoch 8/20
2/2 [==============================] - 1s 286ms/step - loss: 2.8643 - acc: 0.9327 